In [ ]:
!apt install zip

# Imports

In [ ]:
import glob
import joblib
import numpy as np
import PIL
from PIL import Image
import pydicom
import tqdm
import os
import cv2
import zipfile
import pandas as pd
import matplotlib.pyplot as plt

# Get images paths

In [ ]:
# Desired output size.
RESIZED_WIDTH, RESIZED_HEIGHT = 224, 224
EACH_WIDTH, EACH_HEIGHT = RESIZED_WIDTH // 2, RESIZED_HEIGHT // 2

OUTPUT_FORMAT = "jpg"

OUTPUT_DIR = "data_argument_224_224"

In [ ]:
label_num2str = {0: 'powdery_mildew',
                     1: 'scab',
                     2: 'complex',
                     3: 'frog_eye_leaf_spot',
                     4: 'rust'}
    
label_str2num = {'powdery_mildew': 0,
                 'scab': 1,
                 'complex': 2,
                 'frog_eye_leaf_spot': 3,
                 'rust': 4}

In [ ]:
data_dir = "../input/plant-pathology-2021-fgvc8"
!ls {data_dir}

In [ ]:
train_dir = "train_images"
train_paths = glob.glob(f"{data_dir}/{train_dir}/*.jpg")
# test_dir = "test_images"
# test_paths = glob.glob(f"{data_dir}/{test_dir}/*.jpg")
# len(train_paths), len(test_paths)

# Preprocess all data

First declare a bunch of useful functions.

In [ ]:
!mkdir -p {OUTPUT_DIR}/{train_dir}

# Load converted images

Let's test that everything is ok!

In [ ]:
TRAIN_DF = pd.read_csv('../input/pp-csv/PP_one_hot.csv')
TRAIN_DF

In [ ]:
EXTRA_TRAIN = pd.DataFrame(columns=["image", "numerical labels"], dtype=str)

In [ ]:
def concat_img(imgs):
    if len(imgs) < 4:
        return
    output_shape = (EACH_WIDTH, EACH_HEIGHT)
    imgs = [cv2.resize(img, output_shape) for img in imgs]
    upper_img = cv2.vconcat([imgs[0], imgs[1]])
    lower_img = cv2.vconcat([imgs[2], imgs[3]])
    img = cv2.hconcat([upper_img, lower_img])
    return img

In [ ]:
length = len(TRAIN_DF)
indexs = np.random.randint(0, length, size=4)

for count in range(1000):
    imgs = []
    new_num_labels = []
    new_labels = []
    indexs = np.random.randint(0, length, size=4)
    for index in indexs:
        filepath = train_paths[index]
        img = cv2.imread(filepath)
        imgs.append(img)
        labels = TRAIN_DF.iloc[index, 1]
        numerical_labels = TRAIN_DF.iloc[index, 2]
        numerical_labels = numerical_labels.replace('[','')
        numerical_labels = numerical_labels.replace(']','')
        for disease in numerical_labels.split(','):
            if not disease or int(disease) in new_num_labels:
                continue
            new_num_labels.append(int(disease))
        new_labels = [label_num2str[label] for label in new_num_labels]
    img_name = f"{count}.jpg"
    new_img = concat_img(imgs)
    new_label = ' '.join(new_labels)
    new_num_label = str(new_num_labels)
    new_file_path = f"{OUTPUT_DIR}/{train_dir}/{img_name}"
    cv2.imwrite(new_file_path, new_img)
    EXTRA_TRAIN = EXTRA_TRAIN.append({"image":img_name, "labels":new_label, "numerical labels":new_num_label}, ignore_index=True)

In [ ]:
# train_num = len(train_paths)
# for i, train_path in enumerate(train_paths):
#     filename = train_path.split("/")[-1]
#     img = Image.open(train_path)
#     img = img.resize((RESIZED_WIDTH, RESIZED_HEIGHT))
#     img.save(os.path.join(OUTPUT_DIR, train_dir, filename))
#     print(f"Successfully process {i}th image of {train_num} images")

# test_num = len(test_paths)
# for i, test_path in enumerate(test_paths):
#     filename = test_path.split("/")[-1]
#     img = Image.open(test_path)
#     img = img.resize((RESIZED_WIDTH, RESIZED_HEIGHT))
#     img.save(os.path.join(OUTPUT_DIR, test_dir, filename))
#     print(f"Successfully process {i}th image of {test_num} images")

In [ ]:
EXTRA_TRAIN.to_csv(f"{OUTPUT_DIR}/extra_train.csv", index=False)

In [ ]:
!zip -r {OUTPUT_DIR}_resized.zip ./{OUTPUT_DIR}/*

In [ ]:
!rm -r -f ./{OUTPUT_DIR}